## 데이터 양상 파악

### import and load json

In [4]:
import os
import os.path as path

In [5]:
os.getcwd()

'/repo/course/sem21_01/youtube-summarization/src/notebook/CES'

In [6]:
### 기본적인 json read
import json
path = '/repo/course/sem21_01/youtube-summarization/data'
bfly_train = path + '/bfly/train.jsonl'
news_train = path + '/news/train.jsonl'
news_test = path + '/news/test.jsonl'
news_dev = path + '/news/dev.jsonl'

In [8]:
import json
import pandas as pd

### bfly train
bfly_df = pd.read_json(bfly_train, lines=True, encoding="utf-8")
print('shape : {}'.format(bfly_df.shape))
bfly_df.head()

shape : (42803, 5)


,media,id,article_original,abstractive,extractive
0,당진시대,327827480,[당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것...,지난 6일 당진시의회 행정사무감사에서 '합창단이 소리를 작게 낼 것이니 알고 있으라...,"[0, 1, 2]"
1,국제신문,339840364,[미국 메이저리그(MLB)에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있...,LA 에인절스의 최지만이 맹활약을 하여 시즌 타율 0.250에서 0.313으로 올리...,"[0, 1, 3]"
2,기호일보,371071597,"[인천 부영공원 운영 생활야구협회 80여 동호회 팀에 260만 원씩 받아, 국유지로...",16일 부평구와 협회 등에 따르면 부영공원 안에 있는 야구장을 구생활체육야구협회가 ...,"[6, 7, 10]"
3,대구일보,354806783,[대구·경북첨단의료산업진흥재단 의약생산센터는 항암주사제 무균충전 시설을 갖추고 있다...,대구·경북첨단의료산업진흥재단 의약생산센터는 약사법 시행규칙에서 정한 바에 따라 전용...,"[1, 6, 4]"
4,대구신문,347022773,[국내 유통되는 탄산음료 중 식품의약품안전처에 품질인증을 받은 제품이 하나도 없는 ...,식품의약품안전처는 29일 어린이가 즐겨마시는 음료를 대상으로 영양성분을 조사한 결과...,"[2, 3, 10]"


In [9]:
import json
import pandas as pd

## train
news_train_df = pd.read_json(news_train, lines=True, encoding="utf-8")
print('shape : {}'.format(news_train_df.shape))
#print(news_train_df.head())

## val
news_val_df = pd.read_json(news_dev, lines=True, encoding="utf-8")
print('shape : {}'.format(news_val_df.shape))
#print(news_val_df.head())


## test
news_test_df = pd.read_json(news_test, lines=True, encoding="utf-8")
print('shape : {}'.format(news_test_df.shape))
#print(news_test_df.head())


shape : (260697, 5)
shape : (10000, 5)
shape : (10000, 5)


### Make ID as set for join operations

In [28]:
# set - ID 정보를 이용한 데이터 비교
set_bfly_ID = set(bfly_df['id'])
set_news_ID_trn = set(news_train_df['id'])
set_news_ID_dev = set(news_val_df['id'])
set_news_ID_tst = set(news_test_df['id'])

print(len(set_bfly_ID))
print(len(set_news_ID_trn))
print(len(set_news_ID_dev))
print(len(set_news_ID_tst))

42803
260697
10000
10000


### check if id matches

In [29]:
intersect = list(set_bfly_ID & set_news_ID_dev)
intersect[:5]

[330113024, 333787136, 369477633, 352569347, 344748032]

In [33]:
bfly_df[bfly_df['id']==330113024]

,media,id,article_original,abstractive,extractive
39753,대전일보,330113024,[금혜원 Urban Depth DB0036(2011) / 작품 사진=우민아트센터 제...,충북 청주 우민아트센터에서 진행되는 '콘크리토피아(CONCRETOPIA) 가상의 유...,"[0, 3, 2]"


In [31]:
news_val_df[news_val_df['id']==330113024]

,media,id,article_original,abstractive,extractive
1130,대전일보,330113024,[금혜원 Urban Depth DB0036(2011) / 작품 사진=우민아트센터 제...,충북 청주 우민아트센터에서 진행되는 '콘크리토피아(CONCRETOPIA) 가상의 유...,"[0, 3, 2]"


### Check ratio

In [34]:
print( len(set_bfly_ID))

42803


In [35]:
complement1 = set_bfly_ID & set_news_ID_trn
print( len(complement1 ))

34352


In [36]:
complement2 = set_bfly_ID & set_news_ID_dev
print( len(complement2 ))

1318


In [37]:
complement3 = set_bfly_ID & set_news_ID_tst
print( len(complement3 ))

1336


In [38]:
len(complement1)+len(complement2)+len(complement3)

37006

In [39]:
union = set_news_ID_trn | set_news_ID_dev
union = union | set_news_ID_tst

print(len(union))

280697


In [40]:
unseen = list(set_bfly_ID - union)
print(len(unseen))

5797


In [42]:
bfly_df.head()

,media,id,article_original,abstractive,extractive
0,당진시대,327827480,[당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것...,지난 6일 당진시의회 행정사무감사에서 '합창단이 소리를 작게 낼 것이니 알고 있으라...,"[0, 1, 2]"
1,국제신문,339840364,[미국 메이저리그(MLB)에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있...,LA 에인절스의 최지만이 맹활약을 하여 시즌 타율 0.250에서 0.313으로 올리...,"[0, 1, 3]"
2,기호일보,371071597,"[인천 부영공원 운영 생활야구협회 80여 동호회 팀에 260만 원씩 받아, 국유지로...",16일 부평구와 협회 등에 따르면 부영공원 안에 있는 야구장을 구생활체육야구협회가 ...,"[6, 7, 10]"
3,대구일보,354806783,[대구·경북첨단의료산업진흥재단 의약생산센터는 항암주사제 무균충전 시설을 갖추고 있다...,대구·경북첨단의료산업진흥재단 의약생산센터는 약사법 시행규칙에서 정한 바에 따라 전용...,"[1, 6, 4]"
4,대구신문,347022773,[국내 유통되는 탄산음료 중 식품의약품안전처에 품질인증을 받은 제품이 하나도 없는 ...,식품의약품안전처는 29일 어린이가 즐겨마시는 음료를 대상으로 영양성분을 조사한 결과...,"[2, 3, 10]"


In [43]:
unseen = list(unseen)
print(len(unseen))
print(unseen)

5797
[332005377, 343605251, 332005382, 345964550, 332005385, 337117194, 333774875, 330596395, 351666220, 351174701, 336396346, 351666235, 330596411, 352485435, 341803070, 334921794, 350257219, 347013186, 351666248, 354648176, 357793925, 332005511, 346849415, 369819786, 357793931, 332660878, 351010968, 338919582, 332660896, 330072231, 333775017, 351862954, 336396458, 342032558, 332005568, 332005571, 332005573, 359203017, 357794005, 331022553, 333775077, 339181806, 343605489, 343605491, 346849546, 314704142, 314704154, 282296616, 282296622, 333775153, 347898163, 346849588, 338723125, 282296630, 350093627, 344523079, 344392007, 351174985, 344392010, 332661067, 333775180, 338657615, 334627156, 344523096, 371097960, 282296687, 332005747, 314704249, 369590668, 368738701, 337740172, 299008404, 371098009, 336396699, 299008412, 332005792, 338985376, 364183985, 336396735, 358580683, 310641103, 347013589, 342917590, 364184036, 337641959, 334627308, 368181741, 342917623, 279052795, 330007036, 3433

In [45]:
unseen_df = pd.DataFrame()

for x in unseen:
    #print(x)
    unseen_df = unseen_df.append(bfly_df[bfly_df['id'] == x], ignore_index = True)

In [46]:
unseen_df.head()

,media,id,article_original,abstractive,extractive
0,국제신문,332005377,[부산 영도구 대평동과 중구 자갈치시장을 오가다 수요 부족으로 사라진 ‘영도 도선’...,부산 영도구는 대평동과 중구 자갈치시장을 오가다 수요 부족으로 사라진 ‘영도 도선’...,"[7, 0, 3]"
1,대구신문,343605251,"[인공지능과 드론 등 미래 기술을 체험할 수 있는 교육의 장이 열린다., 대구 수성...",4차 산업혁명 시대의 미래 핵심 기술에 대한 체험과 이해를 통해 변화하는 미래 환경...,"[1, 2, 3]"
2,국제신문,332005382,"[- LH, 원도심에 공급 확대 추진, - 구, 청년창업시설 입주도 검토, 부산국토...",동구는 현재 한국토지주택공사(LH)와 국토관리청 이전 부지 활용 계획을 놓고 논의하...,"[7, 8, 10]"
3,남도일보,345964550,"[환경단체 “황룡강 장록습지, 국가습지로 지정해야” 촉구, 22일 세계 생물 다양성...","광주전남녹색연합, 광주환경운동연합 등 13개 단체로 구성된 '장록국가습지 지정을 위...","[2, 3, 5]"
4,국제신문,332005385,"[- 靑비서실 대통령 대면보고 축소, - 장관 등 내각의 보고는 확대, 문재인 대통...",문재인 대통령이 23일 공정경제전략회의를 주재하고 “정부는 대기업 대주주의 중대한 ...,"[2, 3, 5]"


In [47]:
unseen_df.shape

(5797, 5)

In [50]:
unseen_df = unseen_df.to_dict('records')
unseen_df

[{'media': '국제신문',
  'id': 332005377,
  'article_original': ['부산 영도구 대평동과 중구 자갈치시장을 오가다 수요 부족으로 사라진 ‘영도 도선’의 명맥을 잇는 ‘깡깡이 유람선(사진)’이 이르면 오는 3월부터 운항할 전망이다.',
   '23일 오후 영도구 대평동 깡깡이예술마을에서 ‘깡깡이 유람선(영도바다버스) 공청회’가 열렸다.',
   '공청회는 깡깡이예술마을 사업단 주최로 열렸으며 부산시, 영도구, 남항관리사업소, 인근 어촌계와 조선소 관계자가 참석했다.',
   '이 자리에서 구는 깡깡이 유람선이 이르면 오는 3월부터 운항할 수 있도록 지원하겠다고 밝혔다.',
   '구가 한국해양수산연수원에 맡긴 ‘부산남항 내 깡깡이유람선 안전운항 방안에 관한 연구용역’이 이달 내 마무리될 예정이다.',
   '연구용역이 완료되면 남항시설관리사업소에 항만시설 사용허가를 받고, 해경에 유선업 등록을 하면 모든 운항 준비가 끝난다.',
   '깡깡이 유람선은 도시재생 사업의 하나로 2016년부터 추진됐다.',
   '과거 대평동과 자갈치 시장을 오가다 수요 부족 등을 이유로 2008년께 사라진 ‘영도 도선’의 추억을 되살리자는 취지다.',
   '유람선은 대평동에서 출발해 남항을 순회하는 형태로 운항하며 20~30명이 승선할 수 있다.',
   '구 홍성호 도시재생추진단 재생정책팀장은 “오래 공을 들인 만큼 제반 준비가 완료됐다.',
   '유람선이 운항을 시작하면 도시재생의 모범사례로 자리 잡을 것으로 기대한다”고 말했다.',
   '애초 깡깡이 유람선은 2017년 말부터 운항할 예정이었으나 사업 속도가 더뎠다.',
   '지난해 항만시설 사용허가를 내달라고 신청했지만, 남항시설관리사업소가 전문가와 관계기관 의견을 수렴하는 게 미흡했다고 판단해 반려했다.',
   '깡깡이예술마을 사업단은 이날 나온 의견을 토대로 미비점을 보완해 사업에 속도를 낼 생각이다.',
   '이승욱 깡깡이 예술마을 사업단장은 “앞서 시범 운항을 했는

In [51]:
save_path = path+'/bfly/unseen.json'

with open(save_path, 'w', encoding='utf-8') as file:

     file.write(json.dumps(unseen_df, ensure_ascii=False))

In [52]:
save_path = path+'/bfly/unseen.jsonl'

with open(save_path, 'w', encoding='utf-8') as file:

     file.write(json.dumps(unseen_df, ensure_ascii=False))

In [55]:
import json
import pandas as pd

save_path = path+'/bfly/unseen.jsonl'

### check loaded data
unseen_load = pd.read_json(save_path, encoding="utf-8")
print('shape : {}'.format(unseen_load.shape))
unseen_load.head()

shape : (5797, 5)


,media,id,article_original,abstractive,extractive
0,국제신문,332005377,[부산 영도구 대평동과 중구 자갈치시장을 오가다 수요 부족으로 사라진 ‘영도 도선’...,부산 영도구는 대평동과 중구 자갈치시장을 오가다 수요 부족으로 사라진 ‘영도 도선’...,"[7, 0, 3]"
1,대구신문,343605251,"[인공지능과 드론 등 미래 기술을 체험할 수 있는 교육의 장이 열린다., 대구 수성...",4차 산업혁명 시대의 미래 핵심 기술에 대한 체험과 이해를 통해 변화하는 미래 환경...,"[1, 2, 3]"
2,국제신문,332005382,"[- LH, 원도심에 공급 확대 추진, - 구, 청년창업시설 입주도 검토, 부산국토...",동구는 현재 한국토지주택공사(LH)와 국토관리청 이전 부지 활용 계획을 놓고 논의하...,"[7, 8, 10]"
3,남도일보,345964550,"[환경단체 “황룡강 장록습지, 국가습지로 지정해야” 촉구, 22일 세계 생물 다양성...","광주전남녹색연합, 광주환경운동연합 등 13개 단체로 구성된 '장록국가습지 지정을 위...","[2, 3, 5]"
4,국제신문,332005385,"[- 靑비서실 대통령 대면보고 축소, - 장관 등 내각의 보고는 확대, 문재인 대통...",문재인 대통령이 23일 공정경제전략회의를 주재하고 “정부는 대기업 대주주의 중대한 ...,"[2, 3, 5]"
